# Import Library

In [15]:
import numpy as np
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from custommodel import RandomForestRegressor as RF

# Data Set

In [5]:
data = pd.DataFrame(data={
    'year':[2021, 2022, 2023, 2024, 2025],
    'ndvi': [4127.14, 4741.4, 4152.37, 4187.05, 4187.05],
    'nir': [1226, 1508.19, 1234.85, 1254.16, 1254.16],
    'red': [0.54077, 0.516945, 0.540222, 0.537683, 0.537683],
    'area': [456.39, 497.77, 416, 373.61, 332.23]
})

In [6]:
data

,year,ndvi,nir,red,area
0,2021,4127.14,1226.00,0.540770,456.39
1,2022,4741.40,1508.19,0.516945,497.77
2,2023,4152.37,1234.85,0.540222,416.00
3,2024,4187.05,1254.16,0.537683,373.61
4,2025,4187.05,1254.16,0.537683,332.23


# Feature Selection

In [10]:
X = data.drop(columns=['area'])
y = data['area']

In [11]:
X

,year,ndvi,nir,red
0,2021,4127.14,1226.00,0.540770
1,2022,4741.40,1508.19,0.516945
2,2023,4152.37,1234.85,0.540222
3,2024,4187.05,1254.16,0.537683
4,2025,4187.05,1254.16,0.537683


In [12]:
y

0    456.39
1    497.77
2    416.00
3    373.61
4    332.23
Name: area, dtype: float64

# Train Random Forest

In [14]:
rf = RF(n_estimators=100, min_samples_split=5, max_depth=5, verbose=True)
rf.fit(X, y)

Tree 1/100 trained.
Tree 2/100 trained.
Tree 3/100 trained.
Tree 4/100 trained.
Tree 5/100 trained.
Tree 6/100 trained.
Tree 7/100 trained.
Tree 8/100 trained.
Tree 9/100 trained.
Tree 10/100 trained.
Tree 11/100 trained.
Tree 12/100 trained.
Tree 13/100 trained.
Tree 14/100 trained.
Tree 15/100 trained.
Tree 16/100 trained.
Tree 17/100 trained.
Tree 18/100 trained.
Tree 19/100 trained.
Tree 20/100 trained.
Tree 21/100 trained.
Tree 22/100 trained.
Tree 23/100 trained.
Tree 24/100 trained.
Tree 25/100 trained.
Tree 26/100 trained.
Tree 27/100 trained.
Tree 28/100 trained.
Tree 29/100 trained.
Tree 30/100 trained.
Tree 31/100 trained.
Tree 32/100 trained.
Tree 33/100 trained.
Tree 34/100 trained.
Tree 35/100 trained.
Tree 36/100 trained.
Tree 37/100 trained.
Tree 38/100 trained.
Tree 39/100 trained.
Tree 40/100 trained.
Tree 41/100 trained.
Tree 42/100 trained.
Tree 43/100 trained.
Tree 44/100 trained.
Tree 45/100 trained.
Tree 46/100 trained.
Tree 47/100 trained.
Tree 48/100 trained.
T

# Evaluate the Model

In [20]:
mean_absolute_error(y, rf.predict(X))

19.35668833333333

# Save the Model

In [21]:
rf.save_model("./rfmodel.pkl")